In [1]:
import pandas as pd
import numpy as np

In [2]:
ten_day_df = pd.read_csv("callups_10day_tidy.csv")
two_way_df = pd.read_csv("two_way_contracts.csv")
conversions_df = pd.read_csv("two_way_conversions.csv")
sportsradar_df_2024 = pd.read_csv("../raw/gleague_player_season_stats_2024_REG.csv")
sportsradar_df_2023 = pd.read_csv("../raw/gleague_player_season_stats_2023_REG.csv")
sportsradar_df_2022 = pd.read_csv("../raw/gleague_player_season_stats_2022_REG.csv")
sportsradar_df_2021 = pd.read_csv("../raw/gleague_player_season_stats_2021_REG.csv")
sportsradar_df_2019 = pd.read_csv("../raw/gleague_player_season_stats_2019_REG.csv")

ten_day_df.head(), two_way_df.head(), conversions_df.head()

(         date   season         player_name               nba_team  \
 0  2007-01-06  2006-07     Thompson, Dijon          Atlanta Hawks   
 1  2007-01-27  2006-07  Richardson, Jeremy          Atlanta Hawks   
 2  2007-04-04  2006-07     Pinkney, Kevinn         Boston Celtics   
 3  2007-04-02  2006-07       Willis, Kevin       Dallas Mavericks   
 4  2007-01-17  2006-07      Major, Renaldo  Golden State Warriors   
 
                                        contract_type  
 0      Two 10-day contracts (only 1st date is shown)  
 1      Two 10-day contracts (only 1st date is shown)  
 2                                    10-day contract  
 3  10-day contract followed by signing for the re...  
 4                                    10-day contract  ,
       player_name        date        nba_team   season    contract_type
 0      Eli Ndiaye  2025-07-01   atlanta hawks  2025-26  two-way signing
 1    Jacob Toppin  2025-07-01   atlanta hawks  2025-26  two-way signing
 2   Caleb Houston  20

In [3]:
def clean_name(name):
    if isinstance(name, str) and "," in name:
        last, first = name.split(",", 1)
        return first.strip() + " " + last.strip()
    return name.strip() if isinstance(name, str) else name

# Clean name fields
ten_day_df["player_name"] = ten_day_df["player_name"].apply(clean_name)
two_way_df["player_name"] = two_way_df["player_name"].apply(clean_name)
conversions_df["player_name"] = conversions_df["player_name"].apply(clean_name)

# SportsRadar originally uses "full_name"
sportsradar_df_2024 = sportsradar_df_2024.rename(columns={"full_name": "player_name"})
sportsradar_df_2024["player_name"] = sportsradar_df_2024["player_name"].apply(clean_name)
sportsradar_df_2023 = sportsradar_df_2023.rename(columns={"full_name": "player_name"})
sportsradar_df_2023["player_name"] = sportsradar_df_2023["player_name"].apply(clean_name)
sportsradar_df_2022 = sportsradar_df_2022.rename(columns={"full_name": "player_name"})
sportsradar_df_2022["player_name"] = sportsradar_df_2022["player_name"].apply(clean_name)
sportsradar_df_2021 = sportsradar_df_2021.rename(columns={"full_name": "player_name"})
sportsradar_df_2021["player_name"] = sportsradar_df_2021["player_name"].apply(clean_name)
sportsradar_df_2019 = sportsradar_df_2019.rename(columns={"full_name": "player_name"})
sportsradar_df_2019["player_name"] = sportsradar_df_2019["player_name"].apply(clean_name)

In [4]:
contracts_df = pd.concat([ten_day_df, two_way_df, conversions_df], ignore_index=True)
contracts_df.head()


,date,season,player_name,nba_team,contract_type
0,2007-01-06,2006-07,Dijon Thompson,Atlanta Hawks,Two 10-day contracts (only 1st date is shown)
1,2007-01-27,2006-07,Jeremy Richardson,Atlanta Hawks,Two 10-day contracts (only 1st date is shown)
2,2007-04-04,2006-07,Kevinn Pinkney,Boston Celtics,10-day contract
3,2007-04-02,2006-07,Kevin Willis,Dallas Mavericks,10-day contract followed by signing for the re...
4,2007-01-17,2006-07,Renaldo Major,Golden State Warriors,10-day contract


In [5]:
contracts_df["year"] = pd.to_datetime(contracts_df["date"]).dt.year

for df in [sportsradar_df_2024, sportsradar_df_2023, sportsradar_df_2022, sportsradar_df_2021, sportsradar_df_2019]:
    df["year"] = df["season_id"].astype(str).str[:4].astype(int)

all_stats = pd.concat([
        sportsradar_df_2024,
        sportsradar_df_2023,
        sportsradar_df_2022,
        sportsradar_df_2021,
        sportsradar_df_2019],
    ignore_index=True
)

In [6]:
final_df = contracts_df.merge(
    all_stats,
    on=["player_name", "year"],
    how="left"
)
final_df = final_df.sort_values(["year", "player_name"], ascending = False)

In [7]:
final_df.to_csv("combined_gleague_contracts.csv", index=False)
final_df

,date,season,player_name,nba_team,contract_type,year,season_id,season_type,team_id,player_id,...,rebounds,steals,blocks,turnovers,fgm,fga,tpm,tpa,ftm,fta
808,2025-02-19,2024-25,Yuri Collins,Golden State Warriors,10-day contract,2025,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
865,2025-07-01,2025-26,Wendell Moore Jr.,detroit pistons,two-way signing,2025,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
884,2025-07-01,2025-26,Vladislav Goldin,miami heat,two-way signing,2025,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
846,2025-07-01,2025-26,Tyson Etienne,brooklyn nets,two-way signing,2025,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
927,2025-07-01,2025-26,Tristan Vukcevic,washington wizards,two-way signing,2025,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,2007-02-08,2006-07,Jared Reiner,Milwaukee Bucks,Two 10-day contracts followed by signing for t...,2007,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2007-01-31,2006-07,Doug Christie,Los Angeles Clippers,Two 10-day contracts (only 1st date is shown),2007,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2007-01-06,2006-07,Dijon Thompson,Atlanta Hawks,Two 10-day contracts (only 1st date is shown),2007,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2007-01-05,2006-07,Andre Brown,Seattle SuperSonics,Two 10-day contracts followed by signing for t...,2007,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
